<a href="https://colab.research.google.com/github/Nurhanifahamatullah/2018/blob/master/Bismillah_I2_dan_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CLUSTERING ULASAN PENGGUNA APLIKASI POSPAY MENGGUNAKAN METODE LDA SIMILARITY 

#1. Scrapping Data

In [1]:
#1. Proses Scraping (untuk mengambil data)
!pip install google-play-scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 1.1 MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-1.1.0-py3-none-any.whl size=24585 sha256=b30eee87fed6994da21bc768da2b5767cccd44fa4eaa0ad7c4e27989a8d42fed
  Stored in directory: /root/.cache/pip/wheels/f4/fc/13/fc4c30a74f33bdd956248d2a0f022780fce59250a6a2301ad8
Successfully built google-play-scraper


In [2]:
from google_play_scraper import app
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
# scraping seluruh data
from google_play_scraper import Sort, reviews 

result, continuation_token = reviews(
    'com.posindonesia.giropos',
    lang='id',
    country='id',
    sort = Sort.NEWEST,
    count = 8000,
    filter_score_with = None
)



In [4]:
# membuat dataframe
data = pd.DataFrame(np.array(result), columns=['review'])

data = data.join(pd.DataFrame(data.pop('review').tolist()))

data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,9384fee9-2865-4be9-b645-be72f4650fb9,Rhedi R Hidayat,https://play-lh.googleusercontent.com/a-/AOh14...,Mantap Aplikasi Pos nya,5,0,2.5.2,2022-06-15 07:17:03,None,NaT
1,eeadb157-20e1-4951-a1b0-aaa486fa4751,Barry Techno,https://play-lh.googleusercontent.com/a-/AOh14...,Beli barang di aliexpress terus barang udah ny...,1,0,2.5.2,2022-06-15 05:16:10,None,NaT
2,d66bef36-b22c-445d-a28c-cc2852beeed1,Joni Siswanto,https://play-lh.googleusercontent.com/a-/AOh14...,Tambah saldo lwt BRI sudah bisa... Mantapz... ...,5,2,2.5.2,2022-06-15 03:37:17,"Hai Sobat Pospay, Kami mohon maaf atas ketidak...",2021-09-06 01:25:38
3,f4dbe8dd-647f-43c4-aa4f-307d80c69af5,Yanti Suminar,https://play-lh.googleusercontent.com/a/AATXAJ...,Kurang soldo,1,0,None,2022-06-15 01:28:58,None,NaT
4,41f2330a-2cab-42d0-9409-0fe29a84d5d7,Felx x,https://play-lh.googleusercontent.com/a/AATXAJ...,Ribet sumpah,1,0,None,2022-06-14 15:41:47,None,NaT


In [5]:
# mengambil data dengan interval waktu
list = data.values.tolist()
dari = '2021-09-01'
sampai = '2022-06-01'
username = []
at = []
content = []
score = []

for i in list:
  date = i[7].strftime('%Y-%m-%d')
  if date >= dari and date <= sampai:
    username.append(i[1])
    at.append(i[7])
    content.append(i[3])
    score.append(i[4])

datapost = {'userName' : username, 'at' : at, 'content' : content, 'score' : score} 
data = pd.DataFrame(datapost)

In [6]:
len(data)

5021

In [ ]:
data[['userName','at','content','score']].head()

,userName,at,content,score
0,AriefSan,2022-06-01 13:51:39,"Setiap buka login mulu, baru pertama makai mau...",1
1,Fitri Yani Hasibuan,2022-06-01 13:46:06,"Biaya admin mahal, terus harus ada yg tinggal ...",2
2,Xyzet 111,2022-06-01 13:31:57,Aplikasinya sangat ribet,1
3,ahmad taufiq,2022-06-01 08:37:37,Aplikasi yang menguji kesabaran :(,1
4,I.B. Parwatha,2022-06-01 04:57:06,Kecewa,1


In [ ]:
data.tail()

,userName,at,content,score
5018,alip susanto,2021-09-01 06:15:35,"Saya lebih mudah bertransaksi, tks Pos Indones...",5
5019,Iskandar Syafaruddin,2021-09-01 05:43:55,Bayar apa pun tidak perlu keluar rumah,5
5020,Sumardi Samsudin,2021-09-01 04:29:43,Layanan aplikaso ini sangat memudahkan costume...,5
5021,ok batak,2021-09-01 04:09:47,Serba bisa ... Bisa dan mudah...,5
5022,noviliagiyai 023,2021-09-01 02:49:56,Sangat membantu☺,5


In [ ]:
data.isnull().sum()

userName    0
at          0
content     0
score       0
dtype: int64

In [ ]:
data.to_csv('data scraping pospay.csv', index=False)

#2. Text Processing

In [ ]:
#2. Proses Text Preprocessing (untuk membersihkan data)
# Tahapan Case Folding (untuk mengubah semua huruf pada data menjadi huruf kecil semua)

import pandas as pd
import string

df = pd.read_csv('data scraping pospay.csv')
content = df['content']
case_folding = []
for data in content:
  data = data.lower()
  case_folding.append(data)

df['case_folding'] = case_folding
df.head()

In [ ]:
# Case Folding: Lowercase
df['case_folding'] = df['case_folding'].str.lower()

# Remove Punctuation: Menghapus tanda baca
punctuation = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', punctuation))

df['case_folding'] = df['case_folding'].apply(lambda text: remove_punctuation(text))

# Non Alphabet Removal: Remove number and <?>
df['case_folding'] = df['case_folding'].str.replace(r'[^a-zA-Z ]\s?',r'', regex=True)
df.head()

In [ ]:
# Cleansing Data
import string, re

def Cleansing(df):
  #lower text
  df = df.lower()
  
  #hapus punctuation
  remove = string.punctuation
  translator = str.maketrans(remove,' '*len(remove))
  df = df.translate(translator)

  #remove ASCII dan unicode
  df = df.encode('ascii','ignore').decode('utf-8')
  df = re.sub(r'[^\x00-\x7f]', r'', df)

  #remove newline
  df = df.replace('\n',' ')
  return df

df['case_folding'] = df['case_folding'].apply(Cleansing)

df.drop_duplicates(keep=False,inplace=True)

In [ ]:
df.to_csv('data.csv')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import ast
from nltk.tokenize import word_tokenize

slang = open("combined_slang_words.txt", "r")
data = slang.read()
slang_words = ast.literal_eval(data)
slang.close()

def convert_word(df):
  word_tokens = word_tokenize(df)
  for w in word_tokens:
    if w in slang_words.keys():
      word_tokens[word_tokens.index(w)]= slang_words[w]

  #filter using NLTK library append it to a string
  filtered_df = [w for w in word_tokens]
  filtered_df = []

  #Looping throught conditions
  for w in word_tokens:
    filtered_df.append(w.lower())
  return ' '.join(filtered_df)

convert_word = df['case_folding'].apply(lambda x: convert_word(x))
df['convert_word'] = convert_word

In [ ]:
df

In [ ]:
# Stopword Removal

stopword = open("stopword.txt", "r")
data = stopword.read()
stop_words = data.split("\n")
stopword.close()

def stopword_remove(df):
  word_tokens = word_tokenize(df)

  #filter using NLTK library it to a string
  filtered_df = [w for w in word_tokens if not w in stop_words]
  filtered_df = []

  #Looping through conditions
  for w in word_tokens:
    #check tokens against stop words, emoticons and punctuations
    if w not in stop_words and w not in string.punctuation:
      filtered_df.append(w.lower())
  return' '.join(filtered_df)
stopword_remove = df['convert_word'].apply(lambda x: stopword_remove(x))
df['stopword_remove'] = stopword_remove

In [ ]:
df

In [ ]:
!pip install Sastrawi

In [ ]:
#stemming (mengubah susunan kata menjadi bentuk kata dasar)
from Sastrawi.Stemmer.StemmerFactory import  StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem(df):
  word_tokens = word_tokenize(df)

  #filter using NLTK library append it to a string
  filtered_df = []

  #Looping through conditions
  for w in word_tokens:
    kata_dasar = stemmer.stem(w)
    filtered_df.append(kata_dasar)
#   return ' '.join(filtered_df)
  return filtered_df

stemming = df['stopword_remove'].apply(lambda x: stem(x))
df['stemming'] = stemming
df

In [ ]:
#tokenize(split teks to word)

token = []

for w in df.stemming:
    z = ' '.join(w)
    data_token = word_tokenize(z)
    token.append(data_token)

df['tokenizing'] = token

In [ ]:
df.to_csv("Data setelah preprocessing.csv", index = False)

In [ ]:
data_pospay = pd.read_csv("Data setelah preprocessing.csv")
data_pospay.head()

In [ ]:
df = pd.DataFrame(df)
df['stemming'] = df['stemming'].apply(str)

In [ ]:
import ast

def join_text_list(texts):
    texts = ast.literal_eval(texts)
    return ' '.join([text for text in texts])
df["data_join"] = df["stemming"].apply(join_text_list)

In [ ]:
df